In [79]:
import pandas as pd
import jieba
import pyLDAvis 
import pyLDAvis.sklearn
import ipywidgets

In [80]:
data = pd.read_csv('datascience.csv', encoding='gb18030')
data.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


In [81]:
data.shape

(1024, 3)

In [82]:
# 处理单一文本分词
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [83]:
data['content_cutted'] = data.content.apply(chinese_word_cut)

In [84]:
# 查看分词结果
data.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [85]:
# 文本向量化
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# 限定提出数量，提取1000个最重要的特征关键词然后停止
n_features = 1000

In [86]:
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10
                                )
tf = tf_vectorizer.fit_transform(data.content_cutted)

In [87]:
#引入LDA(Latent Dirichlet allocation)
from sklearn.decomposition import LatentDirichletAllocation

In [88]:
n_topics = 5 
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)

In [89]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [90]:
# 提取主题中的若干个关键词
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1: -1]]))
    print()

In [91]:
n_top_words = 20

In [92]:
tf_features_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_features_names, n_top_words)


Topic #0:
学习 模型 使用 算法 方法 机器 可视化 神经网络 特征 处理 计算 系统 不同 数据库 训练 分类 基于 工具 一种 深度
Topic #1:
这个 就是 可能 如果 他们 没有 自己 很多 什么 不是 但是 这样 因为 一些 时候 现在 用户 所以 非常 已经
Topic #2:
企业 平台 服务 管理 互联网 公司 行业 数据分析 业务 用户 产品 金融 创新 客户 实现 系统 能力 产业 工作 价值
Topic #3:
中国 2016 电子 增长 10 市场 城市 2015 关注 人口 检索 30 或者 其中 阅读 应当 美国 全国 同比 20
Topic #4:
人工智能 学习 领域 智能 机器人 机器 人类 公司 深度 研究 未来 识别 已经 医疗 系统 计算机 目前 语音 百度 方面



In [93]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.show(pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer))


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [22/Jun/2017 15:42:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:42:58] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:42:58] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:42:58] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:42:58] code 404, message Not Found
127.0.0.1 - - [22/Jun/2017 15:42:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jun/2017 15:44:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:15] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:15] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:15] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:38] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:38] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2017 15:44:38] "GET /LDAvis.js HTTP/1.1" 200 -
